# 模型训练、评估与推理

- 模型训练：训练包括多轮迭代（epoch），每轮迭代遍历一次训练数据集，并且每次从中获取一小批（mini-batch）样本，送入模型执行前向计算得到预测值，并计算预测值（predict_label）与真实值（true_label）之间的损失函数值（loss）。执行梯度反向传播，并根据设置的优化算法（optimizer）更新模型的参数。观察每轮迭代的 loss 值减小趋势，可判断模型训练效果。
- 模型评估：将测试数据集送入训练好的模型进行评估，得到预测值，计算预测值与真实值之间的损失函数值（loss），并计算评价指标值（metric），便于评估模型效果。
- 模型推理：将待验证的数据（样本）送入训练好的模型执行推理，观察并验证推理结果（标签）是否符合预期。


飞桨框架提供了两种训练、评估与推理的方法：
- 使用飞桨高层 API：先用 paddle.Model 对模型进行封装，然后通过 Model.fit 、 Model.evaluate 、 Model.predict 等完成模型的训练、评估与推理。
- 使用飞桨基础 API：提供了损失函数、优化器、评价指标、更新参数、反向传播等基础组件的实现，可以更灵活地应用到模型训练、评估与推理任务中，当然也可以很方便地自定义一些组件用于相关任务中。

In [1]:
import paddle

In [2]:
# 加载数据集
from paddle.vision.transforms import Normalize

transform = Normalize(mean=[127.5], std=[127.5], data_format='CHW')

train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform)
test_dataset = paddle.vision.datasets.MNIST(mode='test', transform=transform)

In [3]:
# 模型组网
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(1, -1),
    paddle.nn.Linear(784, 512),
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0, 2),
    paddle.nn.Linear(512, 10)
)

## 模型封装

In [4]:
# 封装模型为一个Model实例，便于进行后续的训练、评估与推理
model = paddle.Model(mnist)

## 配置训练准备参数

- 优化器（optimizer）：即寻找最优解的方法，可计算和更新梯度，并根据梯度更新模型参数。飞桨框架在 paddle.optimizer 下提供了优化器相关 API。并且需要为优化器设置合适的学习率，或者指定合适的学习率策略，飞桨框架在 paddle.optimizer.lr 下提供了学习率策略相关的 API。
- 损失函数（loss）：用于评估模型的预测值和真实值的差距，模型训练过程即取得尽可能小的 loss 的过程。飞桨框架在 paddle.nn Loss层 提供了适用不同深度学习任务的损失函数相关 API。
- 评价指标（metrics）：用于评估模型的好坏，不同的任务通常有不同的评价指标。飞桨框架在 paddle.metric 下提供了评价指标相关 API。

In [5]:
model.prepare(optimizer=paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters()),  # 学习率
             loss=paddle.nn.CrossEntropyLoss(),  # 交叉熵损失函数
             metrics=paddle.metric.Accuracy())  # 准确率

## 模型训练

训练过程采用二层循环嵌套方式：
- 内层循环完成整个数据集的一次遍历，采用分批次方式
- 外层循环根据设置的训练轮次完成数据集的多次遍历，因此需要指定至少三个关键参数：训练数据集，训练轮次和每批次大小

In [9]:
model.fit(train_dataset, epochs=5, batch_size=100, verbose=1)  # verbose指定日志格式

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 600/600 [==============================] - loss: 0.0247 - acc: 0.9881 - 5ms/step          
Epoch 2/5
step 600/600 [==============================] - loss: 0.0181 - acc: 0.9895 - 4ms/step          
Epoch 3/5
step 600/600 [==============================] - loss: 0.0328 - acc: 0.9898 - 4ms/step          
Epoch 4/5
step 600/600 [==============================] - loss: 0.0338 - acc: 0.9903 - 4ms/step          
Epoch 5/5
step 600/600 [==============================] - loss: 0.0332 - acc: 0.9903 - 4ms/step          


## 评估模型

训练好模型后，可在事先定义好的验证数据集上，使用 Model.evaluate 接口完成模型评估操作，结束后根据在 Model.prepare 中定义的 loss 和 metric 计算并返回相关评估结果，返回格式是一个字典。

In [12]:
eval_result = model.evaluate(test_dataset, verbose=1)

eval_result

Eval begin...
step 10000/10000 [==============================] - loss: 0.0000e+00 - acc: 0.9787 - 535us/step          
Eval samples: 10000


{'loss': [0.0], 'acc': 0.9787}

## 执行推理

对训练好的模型进行推理验证，只需传入待执行推理验证的样本数据，即可计算并返回推理结果

返回格式是一个列表：
- 模型是单一输出：输出的形状为[1, n]，n表示数据集的样本数，每个列表元素是对应原始数据经过模型计算后得到的预测结果，类型为numpy数组。
- 模型是多输出：输出的形状为[m, n]，m表示标签的种类数，在多标签分类任务中，m会根据标签的数目而定。

In [19]:
# 在测试数据集上执行推理
pre_result = model.predict(test_dataset)

# 这里是单一输出，形状为[1, 10000]，10000是测试数据集的数据量
print(len(pre_result))

# 第一个推理结果，这个数组表示每个数字的预测概率
print(pre_result[0][0])

Predict begin...
step 10000/10000 [==============================] - 644us/step          
Predict samples: 10000
1
[[-13.735558    -2.1772842   -1.9931191   -0.32802913 -16.975414
   -9.507339   -18.08655     18.227259    -7.0612817   -0.03320307]]


In [20]:
# 取出预测值中概率最高的一个的下标，做为预测标签
pre_result[0][0].argmax()

7